In [37]:
import autogen

config_list = autogen.config_list_from_json("OAI_CONFIG_LIST", filter_dict={"model": ["gpt-4"]})

In [38]:
gpt4_api_key = config_list[0]["api_key"]


In [39]:
import os
os.environ['OPENAI_API_KEY'] = gpt4_api_key

In [40]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

## Steps
### 1. Build up a vector store with whitepaper.

In [41]:
loaders = [ PyPDFLoader('./Raft.pdf') ]
docs = []
for l in loaders:
    docs.extend(l.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(docs)

In [42]:
vectorstore = Chroma(
    collection_name="full_documents",
    embedding_function=OpenAIEmbeddings()
)
vectorstore.add_documents(docs)

['9be9e0ac-7bfe-11ee-95ff-841b77a742ba',
 '9be9e0ad-7bfe-11ee-9868-841b77a742ba',
 '9be9e0ae-7bfe-11ee-91d6-841b77a742ba',
 '9be9e0af-7bfe-11ee-8864-841b77a742ba',
 '9be9e0b0-7bfe-11ee-81bd-841b77a742ba',
 '9be9e0b1-7bfe-11ee-97ba-841b77a742ba',
 '9be9e0b2-7bfe-11ee-8b0b-841b77a742ba',
 '9be9e0b3-7bfe-11ee-bb4e-841b77a742ba',
 '9be9e0b4-7bfe-11ee-b5e5-841b77a742ba',
 '9be9e0b5-7bfe-11ee-9947-841b77a742ba',
 '9be9e0b6-7bfe-11ee-8b79-841b77a742ba',
 '9be9e0b7-7bfe-11ee-8528-841b77a742ba',
 '9be9e0b8-7bfe-11ee-b721-841b77a742ba',
 '9be9e0b9-7bfe-11ee-928c-841b77a742ba',
 '9be9e0ba-7bfe-11ee-bb65-841b77a742ba',
 '9be9e0bb-7bfe-11ee-aa79-841b77a742ba',
 '9be9e0bc-7bfe-11ee-bc02-841b77a742ba',
 '9be9e0bd-7bfe-11ee-885d-841b77a742ba',
 '9be9e0be-7bfe-11ee-8d05-841b77a742ba',
 '9be9e0bf-7bfe-11ee-8265-841b77a742ba',
 '9be9e0c0-7bfe-11ee-9524-841b77a742ba',
 '9be9e0c1-7bfe-11ee-acf9-841b77a742ba',
 '9be9e0c2-7bfe-11ee-802a-841b77a742ba',
 '9be9e0c3-7bfe-11ee-bc89-841b77a742ba',
 '9be9e0c4-7bfe-

### 2. Set up a conversational retrieval QA chain by LangChain, based on the vector store.

In [43]:
qa = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0),
    vectorstore.as_retriever(),
    memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True)
)

In [44]:
result = qa(({"question": "What is Raft?"}))

In [45]:
result['answer']

' Raft is a consensus algorithm for replicated state machines that uses a replicated log and leader election to maintain the log and ensure consensus. It is specified in TLA+ and can be implemented and used by developers. The state space is large and abstract.'

### 3. Define a function answer_raft_question

It takes a parameter question, calls the QA chain, and answer it by returning the answer from the chain response.


In [46]:
def answer_raft_question(question):
    response = qa({"question": question})
    return response["answer"]

### 4. Set up AutoGen user agent and assistant agent with function calling enabled.

In [47]:
llm_config={
    "request_timeout": 600,
    "seed": 42,
    "config_list": config_list,
    "temperature": 0,
    "functions": [
        {
            "name": "answer_raft_question",
            "description": "Answer any Raft related questions",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question to ask in relation to Raft protocol",
                    }
                },
                "required": ["question"],
            },
        }
    ],
}

In [48]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "."},
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
    function_map={"answer_raft_question": answer_raft_question}
)

### It's time to start a task for the agents.

Now, let's user the user agent to ask the agents to write an introduction blog for Raft.


In [49]:
# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""
I'm writing a blog to introduce the Raft. Find the answers to the 3 questions below and write an introduction based on them.

1. What is Raft?
2. What are the main differences between Raft and Paxos?
3. How to use Raft?

Start the work now.
"""
)


user_proxy (to assistant):


I'm writing a blog to introduce the Raft. Find the answers to the 3 questions below and write an introduction based on them.

1. What is Raft?
2. What are the main differences between Raft and Paxos?
3. How to use Raft?

Start the work now.


--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested function Call: answer_raft_question *****
Arguments: 

{
  "question": "What is Raft?"
}
*********************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION answer_raft_question...
user_proxy (to assistant):

***** Response from calling function "answer_raft_question" *****
 Raft is a consensus algorithm used for replicated state machines that ensures all nodes in the cluster agree on the same sequence of state transitions. It uses a replicated log to maintain the state of the system and a leader 